In [9]:
import pandas, numpy

import ynet

import requests

import bs4

In [10]:
valid_topic_ids = ['891-317', '109-185']

def _scrape_articles_urls_from_topic(topic_id: str, year: int, month: int):
    if year < 2000:
        year = 2000 + year if year < 30 else 1900 + year
    if month < 10:
        month = '0' + str(month)
    url = f'https://www.ynet.co.il/home/0,7340,L-4269-{topic_id}-{year}{month}-1,00.html'
    html = bs4.BeautifulSoup(requests.get(url).content)
    html = html.find_all(attrs={'class': 'ghciArticleIndex1'})[0].find_all('table')[1]

    articles_htmls = html.find_all(attrs={'class': 'smallheader'})
    articles_urls = ['https://www.ynet.co.il' + article.get('href')
                    for article in articles_htmls]
    return articles_urls

def scrape_articles():
    articles_urls = []
    for topic_id in valid_topic_ids:
        for year in range(2000,2021 + 1):
            for month in range(1,12 + 1):
                try:
                    urls = _scrape_articles_urls_from_topic(topic_id, year, month)
                    assert type(urls) == list
                    articles_urls += urls
                except:
                    print(topic_id, year, month, 'failed')
                    pass
    return list(set(articles_urls))

In [ ]:
articles_urls = scrape_articles()

891-317 2021 7 failed
109-185 2001 8 failed
109-185 2002 4 failed


In [ ]:
def save_list(l):
    with open('article_ids.txt', 'w') as f:
        for item in l:
            f.write("%s\n" % item)
save_list(articles_urls)

In [ ]:
def get_article(article_url: str):
    html = bs4.BeautifulSoup(requests.get(url).content)
    main_title = html.find_all(attrs={'class' : 'mainTitle'})[0].text.strip('"')
    sub_title = html.find_all(attrs={'class' : 'subTitle'})[0].text.strip('"')
    content = html.find_all(attrs={'class' : 'public-DraftEditor-content'})[0].text.strip('"')
    return pd.Series({
        'main_title' : main_title, 
        'sub_title' : sub_title, 
        'content' : content
    })

In [ ]:
urls = pd.Series(articles_urls[:500])
data = urls.apply(get_article)

In [ ]:
content = data['sub_title']
titles = data['main_title']

In [ ]:
X = content.str.split()
y = titles.str.split()
X.sample()

In [45]:
[(content_tokenized.index(word) if word in content_tokenized else -1) for word in main_title_tokenized]

[3, -1, -1, 9, 80, -1, 102, -1, -1, 107, 17, 86, -1]